In [1]:
%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.insert(0, "../scripts")
import util
import camera
from track import Track

np.set_printoptions(precision=4)
!pwd

/home/zp-yang/git/visnet/src/visnet/notebooks


In [2]:
def target_traj_circle(t):
    x = 15*np.sin(2 * np.pi * 0.1 * t / 3)
    y = 10*np.cos(2 * np.pi * 0.1 * t / 3)
    z = 20
    return np.array([x,y,z])

def target_traj_straight(t, start, finish, tf):
    max_dist = np.linalg.norm(start-finish)
    v_max = (finish-start) / tf

    return start + v_max * t

def target_stationary(start):
    return start

In [3]:
# %%timeit
cam_param = [642.0926, 0, 1000.5, 0, 642.0926, 1000.5, 0, 0, 1]
cam_params = [cam_param]*4
# [x y z roll pitch yaw]
cam_poses = np.array([
    [20, 20, 12, 0, 0, -2.2],
    [20, -20, 12, 0, 0, 2.2],
    [-20, -20, 12, 0, 0, 0.7],
    [-20, 20, 12, 0, 0, -0.7],
])

cam_group = camera.CamGroup(cam_params, cam_poses[0:4,:])

n_targets = 2
tracks = []

t_final = 200

verbose = False
n_particles = 1000

x0_1 = np.array([-20, 2, 15])
x0_2 = np.array([20, -2, 15])
# x0_2 = np.array([0, 10, 20])
x0_3 = np.array([5, 20, 15])

track1 = Track(n_particles, x0=x0_1, label=1)
track2 = Track(n_particles, x0=x0_2, label=1)

track3 = None

tracks = [track1, track2]
tracks_true = np.hstack([x0_1, x0_2])

state_est1 = []
state_est2 = []

association_order = []

triag_sol = []

for t in range(t_final):
    target_1 = target_traj_straight(t, x0_1, x0_1+[40,0,0], t_final)
    target_2 = target_traj_straight(t, x0_2, x0_2+[-40,0,0], t_final)

    # target_2 = target_traj_circle(t)
    # target_1 = target_stationary(x0_1)
    # target_2 = target_stationary(x0_2)
    targets = np.vstack([target_1, target_2])
    tracks_true = np.vstack([tracks_true, targets.reshape(1,6)])

    # if t > 100:
    #     target_3 = target_traj_straight(t-100, x0_3, x0_3+[0,-30,0], t_final-50)
    #     targets = np.vstack([targets, target_3])
    #     if track3 is None:
    #         track3 = Track(n_particles, target_3, 0)
    #         tracks.append(track3)

    #each camera can give multiple measurements
    z = cam_group.get_group_measurement(targets, labels= np.array([1, 1]))
    z_triag = cam_group.get_group_measurement(np.vstack([target_1]), labels=np.array([1, ]))
    
    triag_sol.append(cam_group.triangulate(z_triag))

    mean_states = np.array([track.mean_state[0:3] for track in tracks])  
    mean_hypo = cam_group.get_group_measurement(mean_states, labels=np.array([1,1]))

    z_a = []
    msmt_order = []
    for z_m, mean_hypo_m in zip(z, mean_hypo):
        rand = np.random.uniform(0,1)
        # if rand < 0.2: # 20% chance of getting no measurements
        #     z_a_m, order = msmt_association(np.array([[-1,-1,-1]]), mean_hypo_m)
        # else:
        z_a_m, order = util.msmt_association(z_m, mean_hypo_m, sigma=40)
        
        z_a.append(z_a_m)
        msmt_order.append(order)
    association_order.append(msmt_order)

    for i, track in enumerate(tracks):
        msmt = [z_a_m[i] for z_a_m in z_a]
        particles = util.dynamics_d(track.particles)
        hypo = util.observe_fn(cam_group, track.particles)
        weights = util.weight_fn(msmt=msmt, hypo=hypo, sigma=40)
        
        track.update(weights, particles)


In [4]:
triag_sol = np.array(triag_sol)
triag_sol.shape

(200, 3)

In [6]:
plt.rcParams.update({'font.size': 18})
association_order = np.array(association_order)
print(association_order.shape)
association_count = np.sum(association_order, axis=1)
track1_rate = (4 - association_count[:, 0]) / 4.0 
track2_rate = association_count[:, 1]  / 4.0

traj_1 = np.array(track1.trajectory)
traj_2 = np.array(track2.trajectory)

fig2 = plt.figure("Association accuracy and Track Error", figsize=(15,10))
ax1 = fig2.add_subplot(2,2,1)
ax1.plot(track1_rate)
ax1.set_ylim(0, 1.1)
ax1.set_ylabel("Association Accuracy")
ax1.set_title("Track 1")
ax1.grid()

ax2 = fig2.add_subplot(2,2,2)
# ax2 = fig2.add_subplot(2,1,1)
ax2.plot(track2_rate)
ax2.set_ylim(0, 1.1)
ax2.set_ylabel("Association Accuracy")
ax2.set_title("Track 2")
ax2.grid()

ax3 = fig2.add_subplot(2,2,3)
ax3.plot(traj_1-tracks_true[0:-1, 0:3], label=["x","y","z"])
ax3.set_xlabel("frames")
ax3.set_ylabel("position error[m]")
ax3.grid()
ax3.legend()

ax4 = fig2.add_subplot(2,2,4)
# ax4 = fig2.add_subplot(2,1,2)
ax4.plot(traj_2-tracks_true[0:-1, 3:6], label=["x","y","z"])
ax4.set_xlabel("frames")
ax4.set_ylabel("position error[m]")
ax4.grid()
ax4.legend()
# plt.suptitle("Maneuvering Target")

(200, 4, 2)


In [8]:
fig = plt.figure("trajectory", figsize=(10,10))

ax = plt.axes(projection="3d")

ax.plot3D(traj_1[:,0], traj_1[:,1], traj_1[:,2], marker='.', color='b')
ax.plot3D(tracks_true[:, 0], tracks_true[:, 1], tracks_true[:, 2], color='y')

ax.plot3D(traj_2[:,0], traj_2[:,1], traj_2[:,2], marker='.', color='r')
ax.plot3D(tracks_true[:, 3], tracks_true[:, 4], tracks_true[:, 5], color='y')

# traj_3 = np.array(track3.trajectory)
# ax.plot3D(traj_3[:,0], traj_3[:,1], traj_3[:,2], marker='.', color='r')
# # ax.plot3D(tracks_true[:, 3], tracks_true[:, 4], tracks_true[:, 5], marker='+', color='y')

ax.set_xlim3d(-25,25)
ax.set_ylim3d(-25,25)
ax.set_zlim3d(0,25)
ax.set_xlabel('x[m]')
ax.set_ylabel('y[m]')
ax.set_zlabel('z[m]')


Text(0.5, 0, 'z[m]')

In [13]:
fig_triag = plt.figure("trajectory - triangulation result", figsize=(10,10))
ax = plt.axes(projection="3d")
ax.plot3D(traj_1[:,0], traj_1[:,1], traj_1[:,2], marker='.', color='b')
ax.plot3D(tracks_true[:, 0], tracks_true[:, 1], tracks_true[:, 2], color='y')

ax.plot3D(triag_sol[:,0], triag_sol[:,1], triag_sol[:,2], marker='.', color='r')

ax.set_xlim3d(-25,25)
ax.set_ylim3d(-25,25)
ax.set_zlim3d(0,25)
ax.set_xlabel('x[m]')
ax.set_ylabel('y[m]')
ax.set_zlabel('z[m]')

fig_error = plt.figure("position error comparison", figsize=(10,10))
ax1 = fig_error.add_subplot(2,1,1)
ax1.plot(traj_1-tracks_true[0:-1, 0:3], label=["x","y","z"])
ax1.set_xlabel("frames")
ax1.set_ylabel("PF position error[m]")
ax1.grid()
ax1.legend()
ax1.set_ylim(-1.5, 1.5)

ax2 = fig_error.add_subplot(2,1,2)
ax2.plot(triag_sol-tracks_true[0:-1, 0:3], label=["x","y","z"])
ax2.set_xlabel("frames")
ax2.set_ylabel("triagulation position error[m]")
ax2.grid()
ax2.legend()
ax2.set_ylim(-1.5, 1.5)

(-1.5, 1.5)

In [25]:
fig = plt.figure("trajectory-2d", figsize=(10,10))

ax = plt.axes()
traj_1 = np.array(track1.trajectory)
ax.plot(traj_1[:,0], traj_1[:,1], marker='.', color='b', label="track 1")
ax.plot(tracks_true[:, 0], tracks_true[:, 1], color='y', label="track 1 ground truth")
traj_2 = np.array(track2.trajectory)
ax.plot(traj_2[:,0], traj_2[:,1], marker='.', color='r', label="track 2")
ax.plot(tracks_true[:, 3], tracks_true[:, 4], color='c', label="track 2 ground truth")
ax.set_xlim(-25,25)
ax.set_ylim(-25,25)
ax.set_xlabel('x[m]')
ax.set_ylabel('y[m]')
ax.grid()
ax.legend()
ax.set_title("Top-Down View of Trajectories of Two Targets")
# ax.set_title("Top-Down View of a Maneuvering Target")


Text(0.5, 1.0, 'Top-Down View of Trajectories of Two Targets')

In [26]:
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
from matplotlib import cm
# Attaching 3D axis to the figure
fig = plt.figure("Particle visualization", figsize=(20, 10))
ax1 = plt.axes(projection="3d")
# ax1 = fig.add_subplot(1, 2, 1, projection="3d")
# ax2 = fig.add_subplot(1, 2, 2, projection="3d")
ax1.view_init(90,-90)

particles_1 = np.array(track1.particles_hist)
particles_2 = np.array(track2.particles_hist)

weights_1 = np.array(track1.weights_hist)
weights_2 = np.array(track2.weights_hist)

print(particles_1.shape, weights_1.shape)
def update_frame(i):
    # NOTE: there is no .set_data() for 3 dim data...
    x_1 = particles_1[i, :, 0]
    y_1 = particles_1[i, :, 1]
    z_1 = particles_1[i, :, 2]
    w_1 = weights_1[i, :]

    x_2 = particles_2[i, :, 0]
    y_2 = particles_2[i, :, 1]
    z_2 = particles_2[i, :, 2]
    w_2 = weights_2[i, :]
    
    ax1.cla()
    ax1.set_xlim3d(-25,25)
    ax1.set_ylim3d(-25,25)
    ax1.set_zlim3d(0,25)
    ax1.scatter(x_1, y_1, z_1, cmap=cm.turbo, c=w_1, s=2)
    ax1.scatter(x_2, y_2, z_2, cmap=cm.turbo, c=w_2, s=2)
    ax1.set_xlabel('x, m')
    ax1.set_ylabel('y, m')
    ax1.set_zlabel('z, m')
        
    return None


ani = FuncAnimation(
    fig, update_frame, t_final, fargs=[],
    interval=int(1000 / 10), blit=False)
# ani.save("../videos/particles.mp4")
# HTML(ani.to_html5_video())
# plt.close()

(200, 1000, 7) (200, 1000)
